In [ ]:
# TODOS
"""
- Hamiltonian DONE maybe?
- Add variable parameters to hamiltonian DONE
- define ansatz layer DONE
- define observable DONE
- define cost function
- optimization loop
- compute variance of gradient of cost function?
- visualize results
"""

In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
n_qubits = 4
n_layers = 1
dev = qml.device("default.qubit", wires=n_qubits)

In [17]:
# supposedly the hamiltonian for the observable but I'm not sure if this is correct
H_obs = qml.Hamiltonian(
    [1, 1, 1], #guess: giving no coeffs is the same as giving them all 1s??? But now we're multiplying these and I'm not sure if the obs is exponentiated as well
    [qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3), 
     qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3),
     qml.PauliX(0) @ qml.PauliX(1) @ qml.PauliX(2) @ qml.PauliX(3)] #TODO: un-hard code
)
print(H_obs)

  (1) [Z0 Z1 Z2 Z3]
+ (1) [Z0 Z1 Z2 Z3]
+ (1) [X0 X1 X2 X3]


In [58]:
@qml.qnode(dev)
def TFIM(weights):
    for i in range(n_layers):
        t = sum([qml.PauliZ(i) @ qml.PauliZ((i+1) % n_qubits) for i in range(n_qubits)])
        qml.exp(t, 1j * weights[i][0]) # for some reason have to multiply weights by a complex number
        t = sum([qml.PauliX(i) for i in range(n_qubits)])
        qml.exp(t, 1j * weights[i][1])
    return qml.expval(H_obs)

weights = np.random.random(size = [n_layers, 2])
print(qml.draw(TFIM)(weights))

0: ─╭Exp─╭Exp─┤ ╭<𝓗(1.00,1.00,1.00)>
1: ─├Exp─├Exp─┤ ├<𝓗(1.00,1.00,1.00)>
2: ─├Exp─├Exp─┤ ├<𝓗(1.00,1.00,1.00)>
3: ─╰Exp─╰Exp─┤ ╰<𝓗(1.00,1.00,1.00)>


In [ ]:
def cost_fn(param):
    #TODO